In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

/home/vish/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from torch import nn
import torch.nn.functional as F

class Generator(nn.Module):
    def __init__(self, input_size: int = 128, output_size: int = 512):
        super().__init__()

        self.seq = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Linear(256, output_size),
            nn.Tanh()
        )

    def forward(self, input: torch.Tensor):
        return self.seq(input)


class Discriminator(nn.Module):
    def __init__(self, input_size: int = 512):
        super().__init__()

        self.seq = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.LeakyReLU(),
            nn.Linear(256, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 32),
            nn.LeakyReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, input: torch.Tensor):
        return self.seq(input)

In [100]:
class BiLSTMDifferentInputs(nn.Module):
    def __init__(self, input_size_1: int = 512, input_size_2: int = 1):
        super().__init__()
        self.lstm1 = nn.LSTM(input_size_1, 256)
        self.lstm2 = nn.LSTM(input_size_2, 32)


    def forward(self, input_1: torch.Tensor, input_2: torch.Tensor):
        out1 = self.lstm1(input_1)
        out2 = self.lstm2(input_2)
        out = []
        for i in range(len(out1)):
            print(out1[i].size())
            print(hn.size())
            t = torch.cat((out1[i], out2[len(out2)-i-1]), 0)
            out.append(t)
        return out

In [85]:
from torch.utils.data import Dataset

class RandomDataGen(Dataset):
    def __init__(self, input_size: int = 128, num_tensors = 512):
        inputs = []
        for i in range(num_tensors):
            ten = torch.normal(0, torch.ones(1, input_size))
            inputs.append(ten)

        self.len = num_tensors

        self.data = torch.stack(inputs, dim=0)

    def __len__(self):
        return self.len
    
    def __getitem__(self, idx: int):
        return self.data[idx]

In [98]:
from torch.utils.data import DataLoader

def get_gan_outputs():
    G = Generator()
    D = Discriminator()
    LSTM = BiLSTMDifferentInputs()

    dataset = RandomDataGen()
    dl = DataLoader(dataset, batch_size=32, shuffle=True)

    for step, x_list in enumerate(dl):
        g_outputs = []
        d_outputs = []

        for x in x_list:
            out = G(x)
            g_outputs.append(out)
            out = D(out)
            d_outputs.append(out)
            # print(out.squeeze().unsqueeze())
        
        d_outputs.reverse()
        

        g_outputs = torch.stack(g_outputs)
        d_outputs = torch.stack(d_outputs)
        print(d_outputs.size())

        out = LSTM(g_outputs, d_outputs)

        return out


In [99]:
get_gan_outputs()

torch.Size([32, 1, 1])
torch.Size([32, 1, 256])
torch.Size([1, 1, 1])


IndexError: index 31 is out of bounds for dimension 0 with size 1